In [31]:
data = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
"""
data = open('puzzle.data').read()

from enum import Enum

State = Enum('State', 'Low High')

class Module:
    def __init__(self, machine, module_name, connections):
        self._machine = machine
        self._module_name = module_name
        self._connections = connections
        self._input_states = dict()
    
    def wire_up(self, module: 'Module'):
        first_wire_up = not self._input_states
        self._input_states[module] = State.Low
        if first_wire_up:
            for m in self._connections:
                if m not in self._machine.modules:
                    self._machine.modules[m] = Output(self._machine, m, [])
                else:
                    self._machine.modules[m].wire_up(self)
    
    def send(self, state):
        self._machine.receive(self._module_name, state)
    
    def emit(self, state):
        for m in self._connections:
            self._machine.register_signal(state)
            # print(f'{self._module_name} -{state} -> {m}')
            self._machine.modules[m].receive(self, state)

class Output(Module):
    def __init__(self, machine, module_name, connections):
        super().__init__(machine, module_name, connections)
        self._state = None

    def receive(self, module, state):
        self._state = state
    
    @property
    def state(self):
        return self._state

class FlipFlop(Module):
    def __init__(self, machine, module_name, connections):
        super().__init__(machine, module_name, connections)
        self._state = State.Low

    def receive(self, module, state):
        if state == State.High:
            return
        self._state = State.High if self._state == State.Low else State.Low
        self.send(self._state)
    
    @property
    def state(self):
        return self._state

class Conjunction(Module):
    def receive(self, module, state):
        self._input_states[module] = state
        if all(state == State.High for state in self._input_states.values()):
            self.send(State.Low)
        else:
            self.send(State.High)
    
    @property
    def state(self):
        return tuple(self._input_states.values())

class Broadcaster(Module):
    def receive(self, module, state):
        self.send(state)
    
    @property
    def state(self):
        return None

class Machine:
    def __init__(self):
        self._count_states = {State.Low: 0, State.High: 0}
        self.modules = dict()
        self.signals = []
        self.monitor = set()
        self.iterations = 0
    
    def iterate(self):
        self.iterations += 1
        self.signals = []
        self.modules['broadcaster'].send(State.Low)
        self.register_signal(State.Low)
        while self.signals:
            module, state = self.signals.pop()
            self.modules[module].emit(state)

    def receive(self, module, state):
        if module in self.monitor and state == State.High:
            print(f'{self.iterations}: {module} -{state}')
        self.signals.insert(0, (module, state))

    def register_signal(self, state):
        self._count_states[state] += 1

def build_machine(data: str) -> Machine:
    machine = Machine()
    for line in data.splitlines():
        module_str, connections = line.split(' -> ')
        connections = connections.split(', ')
        module_name = module_str if module_str == 'broadcaster' else module_str[1:]
        if module_str.startswith('%'):
            machine.modules[module_name] = FlipFlop(machine, module_name, connections)
        elif module_str.startswith('&'):
            machine.modules[module_name] = Conjunction(machine, module_name, connections)
        elif module_str == 'broadcaster':
            machine.modules[module_name] = Broadcaster(machine, module_name, connections)
    
    machine.modules['broadcaster'].wire_up(None)
    return machine

def solve(data: str) -> int:
    machine = build_machine(data)

    for _ in range(1000):
        machine.iterate()
    return machine._count_states[State.High] * machine._count_states[State.Low]

solve(data)

818723272

In [32]:
def solve2(data: str) -> int:
    machine = build_machine(data)

    rx_driver = next(m for m in machine.modules.values() if 'rx' in m._connections)._module_name
    machine.monitor |= set(m._module_name for m in machine.modules[rx_driver]._input_states)

    for _ in range(5000):
        machine.iterate()

solve2(data)

# multiply the cycles of this four signals together and get the overall cycle length

3917: ch -State.High
3943: gh -State.High
3947: th -State.High
4001: sv -State.High
